In [5]:
import pickle
import pandas as pd
import nltk
from preprocessing import * # import clean function
import json


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xinminaw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/xinminaw/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# instantiate models
DATA_FOLDER = "../data/"
LOGREG_VECT = DATA_FOLDER + "saved_models/relevance_models/model_LR_vectorizer.pkl"
LOGREG_MODEL = DATA_FOLDER + "saved_models/relevance_models/model_LR.pkl"
SVM_VECT = DATA_FOLDER + "saved_models/relevance_models/model_SVM_vectorizer.pkl"
SVM_MODEL = DATA_FOLDER + "saved_models/relevance_models/model_SVM.pkl"
NB_VECT = DATA_FOLDER + "saved_models/relevance_models/model_NB_vectorizer.pkl"
NB_MODEL = DATA_FOLDER + "saved_models/relevance_models/model_NB.pkl"
RF_VECT = DATA_FOLDER + "saved_models/relevance_models/model_RF_vectorizer.pkl"
RF_MODEL = DATA_FOLDER + "saved_models/relevance_models/model_RF.pkl"
LOGREG_MODEL_BERT = DATA_FOLDER + "saved_models/relevance_models/model_LR_BERT.pkl"
META_MODEL = DATA_FOLDER + "saved_models/relevance_models/model_meta.pkl"

In [210]:
# takes in json with 1 field
def modelling_pipeline(company):
    # READ PROCESSED DATA
#     NEW RECORD
#     with open(output_name,) as inputfile:
#         processed_json = json.load(inputfile)
    
    processed_json = company
    sentences_direct = processed_json["bert_relevant_sentences_direct_original"]
    sentences_indirect = processed_json["bert_relevant_sentences_indirect_original"]
    sentences = dict(list(sentences_direct.items()) + list(sentences_indirect.items()))
    
    # for BERT models
    sentences_direct_embeddings = company["bert_relevant_sentences_direct_original_embeddings"]
    sentences_indirect_embeddings  = company["bert_relevant_sentences_indirect_original_embeddings"]
    sentences_embeddings  = dict(list(sentences_direct_embeddings.items()) + list(sentences_indirect_embeddings.items()))

    page = []
    sentence = []
    sentence_embeddings = []
    
    for page_number,page_sentences in sentences.items():
        for sent in page_sentences:
            page.append(page_number)
            sentence.append(sent)

    for page_number,page_sentences in sentences_embeddings.items():
        for sent in page_sentences:
            sentence_embeddings.append(sent)
    
    processed_df = pd.DataFrame({"page":page,"sentence":sentence,"sentence_embeddings":sentence_embeddings})
    processed_df['cleaned_sentence'] = processed_df['sentence'].apply(clean_sentence)
    
    # MAKE PREDICTIONS

    # LOGISTIC REGRESSION PREDICTION
    lr_vectorizer = pickle.load(open(LOGREG_VECT, "rb"))
    lr_model = pickle.load(open(LOGREG_MODEL, "rb"))
    lr_transformed_text = lr_vectorizer.transform(processed_df.cleaned_sentence)
    lr_predictions = lr_model.predict_proba(lr_transformed_text)
    processed_df["LR_prob_1"] = lr_predictions[:, 1]
    processed_df["LR_prob_0"] = lr_predictions[:, 0]

    # SUPPORT VECTOR MACHINE PREDICTION
    svm_vectorizer = pickle.load(open(SVM_VECT, "rb"))
    svm_model = pickle.load(open(SVM_MODEL, "rb"))
    svm_transformed_text = svm_vectorizer.transform(processed_df.cleaned_sentence)
    svm_predictions = svm_model.predict_proba(svm_transformed_text)
    processed_df["SVM_prob_1"] = svm_predictions[:, 1]
    processed_df["SVM_prob_0"] = svm_predictions[:, 0]

    # NAIVE BAYES PREDICTION
    nb_vectorizer = pickle.load(open(NB_VECT, "rb"))
    nb_model = pickle.load(open(NB_MODEL, "rb"))
    nb_transformed_text = nb_vectorizer.transform(processed_df.cleaned_sentence)
    nb_predictions = nb_model.predict_proba(nb_transformed_text)
    processed_df["NB_prob_1"] = nb_predictions[:, 1]
    processed_df["NB_prob_0"] = nb_predictions[:, 0]

    # RANDOM FOREST PREDICTION
    rf_vectorizer = pickle.load(open(RF_VECT, "rb"))
    rf_model = pickle.load(open(RF_MODEL, "rb"))
    rf_transformed_text = rf_vectorizer.transform(processed_df.cleaned_sentence)
    rf_predictions = rf_model.predict_proba(rf_transformed_text)
    processed_df["RF_prob_1"] = rf_predictions[:, 1]
    processed_df["RF_prob_0"] = rf_predictions[:, 0]

    print("LR, SVM, NB, RF predictions complete")
    
    # LOGISTIC REGRESSION BERT
    lr_bert_model = pickle.load(open(LOGREG_MODEL_BERT, "rb"))
#     bert_embeddings = vectorize_dataset_bert(processed_df)
    lr_bert_predictions = lr_bert_model.predict_proba(list(processed_df.sentence_embeddings))
    processed_df["LR_BERT_prob_1"] =  lr_predictions[:, 1]
    processed_df["LR_BERT_prob_0"] =  lr_predictions[:, 0]
    
#     # SUPPORT VECTOR MACHINE BERT
#     svm_bert_model = pickle.load(open(SVM_MODEL_BERT, "rb"))
#     svm_bert_predictions = svm_bert_model.predict_proba(list(processed_df.sentence_embeddings))
#     processed_df["SVM_BERT_prob_1"] = svm_bert_predictions[:, 1]
#     processed_df["SVM_BERT_prob_0"] = svm_bert_predictions[:, 0]
        
    #processed_df.to_csv(prediction_csv, index=False)

    print("BASELINE PREDICTIONS COMPLETE")        

    
    # STACKING MODEL
    # read baseline model predictions 
    predictions_df = processed_df
    
    # generate ensemble predictions
    meta_model = pickle.load(open(META_MODEL, "rb"))

    # fit model
    predictions = meta_model.predict_proba(predictions_df[['RF_prob_0','RF_prob_1','LR_prob_0', 'LR_prob_1',
       'NB_prob_0','NB_prob_1',"LR_BERT_prob_0","LR_BERT_prob_1",'SVM_prob_0', 'SVM_prob_1']])[:, 0:2]
    predictions_df["prob_0"] = 0
    predictions_df["relevance_prob"] = 0 #prob_1
    predictions_df["relevance"] = meta_model.predict(predictions_df[['RF_prob_0','RF_prob_1','LR_prob_0', 'LR_prob_1',
       'NB_prob_0','NB_prob_1',"LR_BERT_prob_0","LR_BERT_prob_1",'SVM_prob_0', 'SVM_prob_1']])
    predictions_df[["prob_0","relevance_prob"]] = predictions

    predictions_df = predictions_df.loc[predictions_df.relevance == 1]
    ensemble_predictions = pd.DataFrame(data=predictions_df,columns=["page", "sentence", "relevance_prob"]) 

    print("ENSEMBLE PREDICTIONS COMPLETE")        
    
    # save ensemble predictions
    return ensemble_predictions
     

# For mass predictions

In [216]:
# for mass predictions
text_json = []
with open(DATA_FOLDER + "preprocessed_sustainability_reports/" + 'all_vfinal_BERT_embeddings.json', 'r') as infile:
    all_data = json.load(infile)
    for company in all_data:
        company_details = {}
        company_details["company"] = company["company"]
        company_details["year"] = company["year"]
        company_details["url"] = company["url"]
        
        sentences_direct = company["bert_relevant_sentences_direct_original"]
        sentences_indirect = company["bert_relevant_sentences_indirect_original"]
        sentences = dict(list(sentences_direct.items()) + list(sentences_indirect.items()))
           
        if len(sentences) != 0:
            data = modelling_pipeline(company)
            data_json = json.loads(data.reset_index().drop("index",axis=1).to_json())
            for k,v in data_json.items():
                data_json[k] = list(data_json[k].values())
            company_details["text_output"] = data_json
        else:
            print("NO RELEVANT SENTENCES")
            company_details["text_output"] = data.reset_index().drop("index",axis=1).to_json()
            
        text_json.append(company_details)
 


LR, SVM, NB, RF predictions complete
BASELINE PREDICTIONS COMPLETE
ENSEMBLE PREDICTIONS COMPLETE


/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


LR, SVM, NB, RF predictions complete
BASELINE PREDICTIONS COMPLETE
ENSEMBLE PREDICTIONS COMPLETE
LR, SVM, NB, RF predictions complete
BASELINE PREDICTIONS COMPLETE
ENSEMBLE PREDICTIONS COMPLETE


/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


LR, SVM, NB, RF predictions complete
BASELINE PREDICTIONS COMPLETE


...

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


LR, SVM, NB, RF predictions complete
BASELINE PREDICTIONS COMPLETE
ENSEMBLE PREDICTIONS COMPLETE
NO RELEVANT SENTENCES


/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


LR, SVM, NB, RF predictions complete
BASELINE PREDICTIONS COMPLETE
ENSEMBLE PREDICTIONS COMPLETE


/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [219]:
with open(DATA_FOLDER + "dashboard_data/" + 'all_text_output.json', "w") as outfile:  
    json.dump(text_json, outfile)

# Pipeline

In [7]:
# takes in json with 1 field
def modelling_pipeline(company):
    # READ PROCESSED DATA
#     NEW RECORD
#     with open(output_name,) as inputfile:
#         processed_json = json.load(inputfile)
    
    processed_json = company
    sentences_direct = processed_json["bert_relevant_sentences_direct_original"]
    sentences_indirect = processed_json["bert_relevant_sentences_indirect_original"]
    sentences = dict(list(sentences_direct.items()) + list(sentences_indirect.items()))
    
    # for BERT models
    sentences_direct_embeddings = company["bert_relevant_sentences_direct_original_embeddings"]
    sentences_indirect_embeddings  = company["bert_relevant_sentences_indirect_original_embeddings"]
    sentences_embeddings  = dict(list(sentences_direct_embeddings.items()) + list(sentences_indirect_embeddings.items()))

    page = []
    sentence = []
    sentence_embeddings = []
    
    for page_number,page_sentences in sentences.items():
        for sent in page_sentences:
            page.append(page_number)
            sentence.append(sent)

    for page_number,page_sentences in sentences_embeddings.items():
        for sent in page_sentences:
            sentence_embeddings.append(sent)
    
    processed_df = pd.DataFrame({"page":page,"sentence":sentence,"sentence_embeddings":sentence_embeddings})
    processed_df['cleaned_sentence'] = processed_df['sentence'].apply(clean_sentence)
    
    # MAKE PREDICTIONS

    # LOGISTIC REGRESSION PREDICTION
    lr_vectorizer = pickle.load(open(LOGREG_VECT, "rb"))
    lr_model = pickle.load(open(LOGREG_MODEL, "rb"))
    lr_transformed_text = lr_vectorizer.transform(processed_df.cleaned_sentence)
    lr_predictions = lr_model.predict_proba(lr_transformed_text)
    processed_df["LR_prob_1"] = lr_predictions[:, 1]
    processed_df["LR_prob_0"] = lr_predictions[:, 0]

    # SUPPORT VECTOR MACHINE PREDICTION
    svm_vectorizer = pickle.load(open(SVM_VECT, "rb"))
    svm_model = pickle.load(open(SVM_MODEL, "rb"))
    svm_transformed_text = svm_vectorizer.transform(processed_df.cleaned_sentence)
    svm_predictions = svm_model.predict_proba(svm_transformed_text)
    processed_df["SVM_prob_1"] = svm_predictions[:, 1]
    processed_df["SVM_prob_0"] = svm_predictions[:, 0]

    # NAIVE BAYES PREDICTION
    nb_vectorizer = pickle.load(open(NB_VECT, "rb"))
    nb_model = pickle.load(open(NB_MODEL, "rb"))
    nb_transformed_text = nb_vectorizer.transform(processed_df.cleaned_sentence)
    nb_predictions = nb_model.predict_proba(nb_transformed_text)
    processed_df["NB_prob_1"] = nb_predictions[:, 1]
    processed_df["NB_prob_0"] = nb_predictions[:, 0]

    # RANDOM FOREST PREDICTION
    rf_vectorizer = pickle.load(open(RF_VECT, "rb"))
    rf_model = pickle.load(open(RF_MODEL, "rb"))
    rf_transformed_text = rf_vectorizer.transform(processed_df.cleaned_sentence)
    rf_predictions = rf_model.predict_proba(rf_transformed_text)
    processed_df["RF_prob_1"] = rf_predictions[:, 1]
    processed_df["RF_prob_0"] = rf_predictions[:, 0]

    print("LR, SVM, NB, RF predictions complete")
    
    # LOGISTIC REGRESSION BERT
    lr_bert_model = pickle.load(open(LOGREG_MODEL_BERT, "rb"))
#     bert_embeddings = vectorize_dataset_bert(processed_df)
    lr_bert_predictions = lr_bert_model.predict_proba(list(processed_df.sentence_embeddings))
    processed_df["LR_BERT_prob_1"] =  lr_predictions[:, 1]
    processed_df["LR_BERT_prob_0"] =  lr_predictions[:, 0]
    
#     # SUPPORT VECTOR MACHINE BERT
#     svm_bert_model = pickle.load(open(SVM_MODEL_BERT, "rb"))
#     svm_bert_predictions = svm_bert_model.predict_proba(list(processed_df.sentence_embeddings))
#     processed_df["SVM_BERT_prob_1"] = svm_bert_predictions[:, 1]
#     processed_df["SVM_BERT_prob_0"] = svm_bert_predictions[:, 0]
        
    #processed_df.to_csv(prediction_csv, index=False)

    print("BASELINE PREDICTIONS COMPLETE")        

    
    # STACKING MODEL
    # read baseline model predictions 
    predictions_df = processed_df
    
    # generate ensemble predictions
    meta_model = pickle.load(open(META_MODEL, "rb"))

    # fit model
    predictions = meta_model.predict_proba(predictions_df[['RF_prob_0','RF_prob_1','LR_prob_0', 'LR_prob_1',
       'NB_prob_0','NB_prob_1',"LR_BERT_prob_0","LR_BERT_prob_1",'SVM_prob_0', 'SVM_prob_1']])[:, 0:2]
    predictions_df["prob_0"] = 0
    predictions_df["relevance_prob"] = 0 #prob_1
    predictions_df["relevance"] = meta_model.predict(predictions_df[['RF_prob_0','RF_prob_1','LR_prob_0', 'LR_prob_1',
       'NB_prob_0','NB_prob_1',"LR_BERT_prob_0","LR_BERT_prob_1",'SVM_prob_0', 'SVM_prob_1']])
    predictions_df[["prob_0","relevance_prob"]] = predictions

    predictions_df = predictions_df.loc[predictions_df.relevance == 1]
    ensemble_predictions = pd.DataFrame(data=predictions_df,columns=["page", "sentence", "relevance_prob"]) 

    print("ENSEMBLE PREDICTIONS COMPLETE")        
    
    # save ensemble predictions
    return ensemble_predictions
     

In [8]:
def relevance_prediction(file_path):
    with open(file_path, 'r') as infile:
        company = json.load(infile)
    
    company_details = {}
    company_details["company"] = company["company"]
    company_details["year"] = company["year"]
    company_details["url"] = company["url"]

    sentences_direct = company["bert_relevant_sentences_direct_original"]
    sentences_indirect = company["bert_relevant_sentences_indirect_original"]
    sentences = dict(list(sentences_direct.items()) + list(sentences_indirect.items()))

    if len(sentences) != 0:
        data = modelling_pipeline(company)
        data_json = json.loads(data.reset_index().drop("index",axis=1).to_json())
        for k,v in data_json.items():
            data_json[k] = list(data_json[k].values())
        company_details["text_output"] = data_json
    else:
        print("NO RELEVANT SENTENCES")
        company_details["text_output"] = data.reset_index().drop("index",axis=1).to_json()
    
    output_path = file_path[:-21] + "_text_output.json"
    
    with open(output_path, "w") as outfile:  
        json.dump(company_details, outfile)
    
    return output_path

In [9]:
file_path = DATA_FOLDER + "sustainability_reports/new/" + 'Canada Pension2017_BERT_embeddings.json'
output_path = relevance_prediction(file_path)


LR, SVM, NB, RF predictions complete
BASELINE PREDICTIONS COMPLETE
ENSEMBLE PREDICTIONS COMPLETE


/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
output_path

'../data/sustainability_reports/new/Canada Pension2017_text_output.json'

In [12]:
with open(output_path, 'r') as infile:
    company = json.load(infile)

In [14]:
file_path

'../data/sustainability_reports/new/Canada Pension2017_BERT_embeddings.json'